# 目录

- [1 - 背景](#1)
- [2 - 理解需求价格弹性](#2)
- [3 - 导入库](#3)
- [4 - 探索性数据分析](#4)
- [5 - 合并数据](#5)
     - [5.1 - 探索合并数据](#5.1)
     - [5.2 - 通过可视化揭示数据侧面](#5.2)
     - [5.3 - 合并数据的探索性分析](#5.3)
- [6 - 建模](#6)
- [7 - 在所有产品上应用模型](#7)
     - [7.1 - 最大利润的最优价格](#7.1)
     - [7.2 - 所有商品的最大利润最优价格](#7.2)
- [8 - 结论](#8)

<a name='1'></a>
### 基于需求价格弹性的价格优化

**价格优化** 是公司使用分析来了解客户如何响应其产品或服务的价格(购买模式),并用于确定在线或线下商店应设定什么价格,以便公司能够满足其大部分目标。一些目标可能是:

获得更多利润、更多收入等
实现这一目标的数据可能多种多样,可以是调查数据、以前的销售数据等

**使用案例** 零售、航空、银行、酒店、本地

**谁需要定价?** 分析师或企业主

**产品定价策略?**

- 成本加成定价(最简单的形式):假设你以$\$100$的价格销售某物,并决定要获得$\$10$的利润,那么你的售价是$\$110$。你根据总采购价格、维护价格、生产价格来决定,即无论成本是多少,你都会在其上添加一个百分比。
- 基于竞争的定价:根据竞争对手的定价,你决定可以降低一定的百分比。
- 基于感知价值的定价:了解特定产品对客户有什么价值,他们如何感知这些产品并相应地定价,例如许多奢侈品,其价格不是基于材料成本、生产成本、分销成本或竞争对手的定价。只是对客户而言具有更高的价值,因此价格更高。
- 基于需求的定价:根据产品的需求,你可以决定定价,即如果需求高,你可以提高价格,如果需求低,你可以进行一些促销或折扣或降低价格。
- 价格弹性

**我应该如何给产品定价** 取决于许多问题和最终目标,我是否想增加销售数量,即增加你销售的商品数量,利润和收入不在你脑海中的首要位置,还是你想要更多的客流量进入你的商店/网站,即每当亚马逊想要打入市场时就会大幅折扣,或者你想增加更多的收入和更多的利润(这是我们感兴趣的)。

**如何使用我拥有的销售数据为产品定价** 
- 销售数据
- 属性 - 价格弹性:衡量商品需求量或供应量对价格变化的响应程度。它可以是,
  - 弹性产品 - 消费者对价格变化有响应(模式基于价格变化)这是我们关注的重点。
  - 非弹性产品 - 消费者对价格变化没有响应(销售不太受价格变化影响,即奢侈品。)

<a name='2'></a>
### 价格弹性

需求价格弹性(PED)是一种衡量指标,用于显示当其他条件不变时,商品或服务的需求量对价格变化的响应程度,即需求与价格之间的关系。即对某物的有效需求随着价格变化而变化。它给出了价格增加1%时数量的百分比变化
数学上
$$ e = \frac{\Delta Q}{\Delta P}$$

我们将查看销售汉堡的咖啡馆的销售情况,咖啡馆老板想知道为其商品设定的最优价格,以获得最大利润。注意:如果价格高,销售会减少,如果价格低,销售会增加,因此总利润会减少。所以这里的任务是找到一个最佳点,能给我们最大的利润。因此我们试图找到数量和价格之间的线性关系,因此在这种情况下使用线性回归

<a name='3'></a>
### 加载库和数据集

In [ ]:
# 导入库
import pandas as pd 
import numpy as np 
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)

In [ ]:
# 在同一单元格中获取多个输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 忽略所有警告
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

# 显示数据框的所有行和列而不是截断版本
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# 加载数据
sold = pd.read_csv("C:\\Python code\\Cafe analysis\\Cafe+-+Sell+Meta+Data.csv")
transaction = pd.read_csv("C:\\Python code\\Cafe analysis\\Cafe+-+Transaction+-+Store.csv")
date_info = pd.read_csv("C:\\Python code\\Cafe analysis\\Cafe+-+DateInfo.csv")

<a name='4'></a>
### 探索性数据分析

**销售数据集:**

SELL_ID: 分类变量,产品中包含的商品组合的标识符。

SELL_CATEGORY: "0"标识单个产品;"2"类别标识组合产品。

ITEM_ID: 分类变量,产品中包含的商品的标识符,与item_name为一对一关系。

ITEM_NAME: 分类变量,标识商品的名称

In [ ]:
# 销售数据集的探索性数据分析
sold.head()
sold.dtypes
sold.describe()
sold.describe(include=["O"])
sold[sold.isnull().any(axis=1)]
sns.pairplot(sold)

# 获取每行的唯一销售ID
pd.concat([sold.SELL_ID, pd.get_dummies(sold.ITEM_NAME)], axis=1)
pd.concat([sold.SELL_ID, pd.get_dummies(sold.ITEM_NAME)], axis=1).groupby(sold.SELL_ID).sum()#seaborn函数，对角线为直方图，其余为散点图

**交易数据集:**

重要说明:假设该产品在当天的价格不会变化。

详细信息:
CALENDAR_DATE: 日期/时间变量,时间始终设置为00:00 AM。

PRICE: 数值变量,与SELL_ID标识的产品价格相关联。

QUANTITY: 数值变量,与SELL_ID标识的产品的销售数量相关联。

SELL_ID: 分类变量,销售产品的标识符。

SELL_CATEGORY: 分类变量,销售产品的类别。

- 可以很容易地看出价格分布是双峰分布,因为它在两个点达到峰值。
- 它可能是许多分布的组合。

In [ ]:
# 交易数据集的探索性数据分析
transaction.head()
transaction.describe()
transaction.describe(include=['O'])
transaction[transaction.isnull().any(axis=1)]
plt.hist(transaction.PRICE)
sns.pairplot(transaction)

In [ ]:
# 日期信息数据集的探索性数据分析
date_info.head()
date_info.describe()
date_info.describe(include=['O'])
date_info.dtypes
date_info[date_info.isnull().any(axis=1)]

# 替换缺失值
date_info['HOLIDAY'] = date_info['HOLIDAY'].fillna("No Holiday")
date_info
sns.pairplot(date_info)

# 进一步的探索性数据分析
np.unique(date_info['HOLIDAY'])
date_info['HOLIDAY'].value_counts()
date_info['CALENDAR_DATE'].min()
date_info['CALENDAR_DATE'].max()
date_info.shape
date_info[date_info.isnull().any(axis=1)]

<a name ='5'></a>
### 合并数据

In [ ]:
#合并并删除不必要的列
merge_data1 = pd.merge(sold.drop(['ITEM_ID'], axis=1), transaction.drop(['SELL_CATEGORY'], axis=1), on= 'SELL_ID')
merge_data1.head(20)

# 使 merge_data1 在所有返回特征的组合上具有唯一性
unique_merge_data1 = merge_data1.groupby(['SELL_ID', 'SELL_CATEGORY', 'ITEM_NAME', 'CALENDAR_DATE', 'PRICE']).QUANTITY.sum()
unique_merge_data1.head(20)
merge_data1.shape
unique_merge_data1.shape
intermediate_data = unique_merge_data1.reset_index()
intermediate_data.head()
intermediate_data['CALENDAR_DATE'].min()
intermediate_data['CALENDAR_DATE'].max()

# 整合日期信息数据
combined_data = pd.merge(intermediate_data, date_info, on= 'CALENDAR_DATE')
combined_data.head()
combined_data.shape
combined_data[combined_data.isnull().any(axis=1)]
np.unique(combined_data.HOLIDAY)
np.unique(combined_data.IS_WEEKEND)
np.unique(combined_data.IS_SCHOOLBREAK)

#这些属性在我们的销售中非常重要，但与价格无关。例如，如果遇到节假日，销售量会增加；
# 如果遇到学校假期且周末，销售量也会增加。但我们不希望看到这种影响，因为我们只对价格的影响感兴趣。
# 数据集中将移除所有与价格无关的销售影响因素。
bau_data = combined_data[(combined_data['HOLIDAY']=='No Holiday') & (combined_data['IS_SCHOOLBREAK']==0) & 
                         (combined_data['IS_WEEKEND']==0)]
bau_data.head()
bau_data.shape
np.unique(bau_data.HOLIDAY)
np.unique(bau_data.IS_WEEKEND)
np.unique(bau_data.IS_SCHOOLBREAK)
bau_data[bau_data['IS_WEEKEND']==1]
bau_data[bau_data['HOLIDAY']!='No Holiday']

<a name='5.1'></a>
#### 组合数据的探索性数据分析

In [ ]:
# 物品名称
plt.hist(bau_data.ITEM_NAME)
# 价格
plt.hist(bau_data.PRICE)
# 组合数据集中的价格与数量
plt.scatter(combined_data.PRICE, combined_data.QUANTITY)
# 在bau数据集中，这不包含特殊日期
plt.scatter(bau_data.PRICE, bau_data.QUANTITY)
# 组合数据的配对图
sns.pairplot(combined_data[['PRICE','QUANTITY','ITEM_NAME']], hue = 'ITEM_NAME', plot_kws={'alpha':0.1})
# bau数据的配对图
sns.pairplot(bau_data[['PRICE','QUANTITY','ITEM_NAME']], hue = 'ITEM_NAME', plot_kws={'alpha':0.1})

价格密度图呈现双峰形态。从图表中可以看出，对于所有数量而言，随着价格的提高，销售量呈下降趋势。尽管可乐在该视图中被隐藏了。我们可以继续计算这些产品的价格弹性。

<a name='5.2'></a>
#### 用可视化手段揭示数据的各个层面

从散点图中可以明显看出，一定存在不同类型的小吃堡在销售。现在让我们看看当用SELL_ID进行区分时的相同分布情况，SELL_ID表示该小吃堡是否为套餐的一部分，因此必须单独处理。

* 因此，SELL_ID 也是导致这些不同群体和不同行为的因素之一，所以按每个 ITEM_NAME 对数据进行拆分是合理的。因为如帖子中所示，我们无法用具有两种分布的数据来拟合模型。

In [ ]:
# 汉堡的各个部分
burger = combined_data[combined_data['ITEM_NAME'] == 'BURGER']
burger.head()
burger.shape 
burger.describe()
sns.scatterplot(x=burger.PRICE, y=burger.QUANTITY)

# alpha用于设置透明度，图例可能需要完整显示以查看各种SELL_ID和组合
burger = combined_data[combined_data['ITEM_NAME'] == 'BURGER']
sns.scatterplot(data = burger, x = burger.PRICE, y = burger.QUANTITY , hue = 'SELL_ID', legend=False, alpha = 0.1)

np.unique(combined_data.SELL_ID)
np.unique(combined_data.SELL_CATEGORY)

# 为特定销售ID创建包含汉堡的组合产品的视觉效果
burger_1070 = combined_data[(combined_data['ITEM_NAME'] == 'BURGER') & (combined_data['SELL_ID'] == 1070)]
burger_1070.head()
burger_1070.describe()
sns.scatterplot(data = burger_1070, x = burger_1070.PRICE, y = burger_1070.QUANTITY, alpha = 0.1)

<a name='6'></a>
### 建模

* 由于R平方值较低，模型与数据的拟合效果不佳，存在较大差异。让我们在移除节假日、周末和学校假期后的数据中进行观察。

* 图表中部分右侧区域的缺失对应特殊日子和节假日。

In [ ]:
# 使用组合数据
burger_model = ols("QUANTITY ~ PRICE", data=burger_1070).fit() 
print(burger_model.summary())
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_partregress_grid(burger_model, fig=fig)

In [ ]:
# 按项目名称“汉堡”对bau数据集进行子集划分
burger = bau_data[bau_data['ITEM_NAME'] == 'BURGER']
burger.head()
burger.shape
burger.describe()
sns.scatterplot(x = burger.PRICE, y = burger.QUANTITY)

burger = bau_data[bau_data['ITEM_NAME'] == 'BURGER']
sns.scatterplot(data=burger, x =burger.PRICE, y= burger.QUANTITY, hue= 'SELL_ID', legend=False, alpha=0.1)

# 唯一销售ID，该BAU数据包含所有销售ID
np.unique(bau_data.SELL_ID)
np.unique(bau_data.SELL_CATEGORY)

# 子集销售编号1070汉堡及套餐
burger_1070 = bau_data[(bau_data['ITEM_NAME'] == 'BURGER') & (bau_data['SELL_ID'] == 1070)]
burger_1070.head()
burger_1070.describe()
sns.scatterplot(data= burger_1070, x =burger_1070.PRICE, y= burger_1070.QUANTITY, alpha=0.1)

# 在新数据上拟合OLS模型
burger_model = ols('QUANTITY ~ PRICE', data=burger_1070).fit()
print(burger_model.summary())
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_partregress_grid(burger_model, fig=fig)

* 由于R平方值的增加，变异程度有所降低，模型对趋势的捕捉也更好。然而，我们可以观察到两条独立的趋势线。

* 我们将再次探索数据，看看数据中还有哪些其他因素可以用来进一步优化我们的模型。

* 图表更加清晰，只包含一个组别。

* 趋势明显，线条清晰，仅有小幅波动。我们可以有把握地说，在这家特定的咖啡馆中，对于售价ID为1070的汉堡，价格系数约为-5.3513，因此价格弹性为-5。

* 现在我们已经弄清楚如何为售价ID为1070的特定汉堡计算价格弹性，接下来我们将将其扩展到其他商品。

In [ ]:
# 再读数据
bau_data.head()

# 不确定“outdoor”是什么意思，也把“outdoor”过滤掉
bau2_data = combined_data[(combined_data['HOLIDAY'] == 'No Holiday') & (combined_data['IS_SCHOOLBREAK'] ==0) & 
                          (combined_data['IS_WEEKEND'] == 1) & (combined_data['IS_OUTDOOR'] ==1)]

# 使用汉堡数据集创建组合
burger_1070 = bau2_data[(bau2_data['ITEM_NAME'] == 'BURGER') & (bau2_data['SELL_ID'] == 1070)]
burger_1070.head()
burger_1070.describe()
sns.scatterplot(data= burger_1070, x = burger_1070.PRICE, y= burger_1070.QUANTITY, alpha= 0.1)

# 使用这些数据创建模型
burger_model = ols("QUANTITY ~ PRICE", data= burger_1070).fit()
print(burger_model.summary())
fig = plt.figure(figsize=(12, 8))
fig = sm.graphics.plot_ccpr(burger_model, "PRICE")

# 让我们从另一个角度看待同一个模型
fig = plt.figure(figsize=(12, 8))
fig = sm.graphics.plot_regress_exog(burger_model, "PRICE", fig=fig)

<a name='7'></a>
### 将模型应用于所有产品

* 这个图表看起来还不错，因为它没有显得杂乱无章。

* 再次看R平方值仍然很低，尽管它捕捉到了下降趋势，但我们仍然看到很多变异性。

* 散点图不是很清晰，存在两个不同的分组，让我们尝试拟合一个模型，我们看到的R平方值比汉堡模型更好，但仍然较低，尽管捕捉到了下降趋势。

* 看起来有两个不同的分组，存在一些价格差距。

* 价格的系数为-5，这意味着价格弹性为-5。

* 与之前的饮料（可乐和咖啡）类似。

* 看起来R平方值比汉堡更好，价格弹性为-2.5，然而与其他所有商品存在的相同问题依然存在。

* 我们将使用更简洁的数据函数，并创建一个通用函数，该函数以数据为输入并产生最佳结果。

In [ ]:
# 只是核对一下卖品ID为2051的汉堡和套餐
burger_2051 = combined_data[(combined_data['ITEM_NAME'] == 'BURGER') & (combined_data['SELL_ID'] == 2051)]
burger_2051.head()
burger_2051.describe()
sns.scatterplot(data= burger_2051, x = burger_2051.PRICE, y= burger_2051.QUANTITY, alpha=0.1)

burger_model = ols('QUANTITY ~ PRICE', data=burger_2051).fit()
print(burger_model.summary())
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_partregress_grid(burger_model, fig=fig)

# look at coke
coke = combined_data[combined_data['ITEM_NAME'] == 'COKE']
coke.head()
coke.shape
coke.describe()
sns.scatterplot(x = coke.PRICE, y= coke.QUANTITY, alpha=0.1)

coke_model = ols("QUANTITY ~ PRICE", data=coke).fit()
print(coke_model.summary())
fig = plt.figure(figsize=(12, 8))
fig = sm.graphics.plot_partregress_grid(coke_model, fig=fig)
fig = plt.figure(figsize=(12, 8))
fig = sm.graphics.plot_regress_exog(coke_model, "PRICE", fig = fig)

# Coffee
coffee = combined_data[combined_data['ITEM_NAME'] == 'COFFEE'] 
coffee.head()
coffee.shape
coffee.describe()
sns.scatterplot(x = coffee.PRICE, y = coffee.QUANTITY , alpha = 0.1)

# 构建和拟合模型
model = ols("QUANTITY ~ PRICE", data=coffee).fit() 
print(model.summary())
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_partregress_grid(model, fig=fig)
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(model, 'PRICE', fig=fig)

# Lemonade柠檬水 
Lemonade_df = combined_data[combined_data['ITEM_NAME'] == 'LEMONADE']
Lemonade_df.head()
Lemonade_df.shape
Lemonade_df.describe()
sns.scatterplot(x = Lemonade_df.PRICE, y = Lemonade_df.QUANTITY , alpha = 0.1)

# fit model for lemonade
model = ols("QUANTITY ~ PRICE", data=Lemonade_df).fit()
print(model.summary())
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_partregress_grid(model, fig=fig)
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(model, 'PRICE', fig=fig)

In [ ]:
# 为所有项目查找弹性
elasticities = {}

# 获取价格弹性的通用函数
def create_model_and_find_elasticity(data):
    model = ols("QUANTITY ~ PRICE", data).fit()
    price_elasticity = model.params[1]
    print('Price elasticiy of the product: ' + str(price_elasticity))
    print(model.summary())
    fig = plt.figure(figsize=(12, 8))
    fig = sm.graphics.plot_partregress_grid(model, fig=fig)
    return price_elasticity, model

In [ ]:
# 应用函数
price_elasticity, model_burger_1070 = create_model_and_find_elasticity(burger_1070)
elasticities['burger_1070'] = price_elasticity

# 2051及其他具有唯一销售ID的汉堡
burger2051_data = bau2_data[(bau2_data['ITEM_NAME'] == "BURGER") & (bau2_data['SELL_ID'] == 2051)]
elasticities['burger_2051'], model_burger_2051 = create_model_and_find_elasticity(burger2051_data)

# 汉堡 2052
burger2052_data = bau2_data[(bau2_data['ITEM_NAME'] == "BURGER") & (bau2_data['SELL_ID'] == 2052)]
elasticities['burger_2052'], model_burger_2052 = create_model_and_find_elasticity(burger2052_data)

# 汉堡 2053
burger2053_data = bau2_data[(bau2_data['ITEM_NAME'] == "BURGER") & (bau2_data['SELL_ID'] == 2053)]
elasticities['burger_2053'], model_burger_2053 = create_model_and_find_elasticity(burger2053_data)

In [ ]:
# coke data
coke_data = bau2_data[bau2_data['ITEM_NAME'] == "COKE"]
create_model_and_find_elasticity(coke_data)

# 2053
coke_data_2053 = bau2_data[(bau2_data['ITEM_NAME'] == "COKE") & (bau2_data['SELL_ID'] == 2053)]
elasticities['coke_2053'], model_coke_2053 = create_model_and_find_elasticity(coke_data_2053)

# 2051
coke_data_2051 = bau2_data[(bau2_data['ITEM_NAME'] == "COKE") & (bau2_data['SELL_ID'] == 2051)]
elasticities['coke_2051'], model_coke_2051 = create_model_and_find_elasticity(coke_data_2051)

# Lemonade sell id 2052
lemonade_data_2052 = bau2_data[(bau2_data['ITEM_NAME'] == "LEMONADE") & (bau2_data['SELL_ID'] == 2052)]
elasticities['lemonade_2052'], model_lemonade_2052 = create_model_and_find_elasticity(lemonade_data_2052)

# coffee with sell id 2053
coffee_data_2053 = bau2_data[(bau2_data['ITEM_NAME'] == "COFFEE") & (bau2_data['SELL_ID'] == 2053)]
elasticities['coffee_2053'], model_coffee_2053 = create_model_and_find_elasticity(coffee_data_2053)

#以表格形式列出弹性
elasticities

<a name='7.1'></a>
#### 实现最大利润的最佳价格

* 让我们以可乐为例，由于我们不知道可乐的购买价格，假设它略低于数据集中可乐的最低价格。


让我们将可乐的购买价格设为9。现在我们希望设定可乐的售价以获得最大利润。此处PRICE表示 selling price（售价）。


$$cokedata.PROFIT = (cokedata.PRICE - buyingpricecoke)* cokedata.QUANTITY$$


* 让我们查看不同价格点的利润情况

* 蓝色线表示价格与数量的关系，橙色线表示价格与利润的关系，我们需要在橙色曲线上找到最大值点

* 让我们找出能获得最大利润的确切价格。

In [ ]:
coke_data.PRICE.min()
coke_data.PRICE.max()

buying_price_coke = 9

# 随机的价格点，这些价格点是合理的
start_price = 9.5
end_price = 20

# 测试数据集
test = pd.DataFrame(columns= [ "PRICE", "QUANTITY"])

# 价格以0.01为步长递增，面额也按此方式增加
test["PRICE"] = np.arange(start_price, end_price, 0.01)

# 2051年的数量
test["QUANTITY"] = model_coke_2051.predict(test["PRICE"])
test 

# 利润
test['PROFIT'] = (test["PRICE"] - buying_price_coke) * test['QUANTITY'] 
test 

# 上面的表格形式
plt.plot(test['PRICE'], test['QUANTITY'])
plt.plot(test['PRICE'], test['PROFIT'])
plt.show()

# 确定实现最大利润的指数点
ind = np.where(test['PROFIT'] == test['PROFIT'].max())[0][0]

#行索引，这仅会在特定某一天产生利润。这将有助于在正常日进行一些库存规划，
# 不考虑节假日等特殊日子。
test.loc[[ind]]

In [ ]:
# 用于最优价格的通用函数
# 定义一个用于寻找最优价格的函数
def find_optimal_price(data, model, buying_price):
    start_price = data.PRICE.min() - 1              # start price
    end_price = data.PRICE.min() + 10               # end price
    test = pd.DataFrame(columns = ["PRICE", "QUANTITY"])  # 选择所需的列
    test['PRICE'] = np.arange(start_price, end_price,0.01)
    test['QUANTITY'] = model.predict(test['PRICE'])         # make predictions
    test['PROFIT'] = (test["PRICE"] - buying_price) * test["QUANTITY"]
    plt.plot(test['PRICE'],test['QUANTITY'])       # 绘制结果
    plt.plot(test['PRICE'],test['PROFIT']) 
    plt.show()
    ind = np.where(test['PROFIT'] == test['PROFIT'].max())[0][0]
    values_at_max_profit = test.iloc[[ind]]
    return values_at_max_profit

<a name='7.2'></a>
#### 计算所有产品的最优价格并以表格形式列出

* 这款特定的1070汉堡的利润曲线和销量曲线没有交叉，因此单独销售时很可能能获得较高的利润。

In [ ]:
# 根据合并数据的价格和数量初始散点图，确定了最优价格，并将最低购买价格设为9。
optimal_price = {}
buying_price = 9

optimal_price['burger_1070'] = find_optimal_price(burger_1070, model_burger_1070, buying_price)
optimal_price
optimal_price['burger_2051'] = find_optimal_price(burger2051_data, model_burger_2051, buying_price)
optimal_price['burger_2052'] = find_optimal_price(burger2052_data, model_burger_2052, buying_price)
optimal_price['burger_2053'] = find_optimal_price(burger2053_data, model_burger_2053, buying_price)
optimal_price['coke_2051'] = find_optimal_price(coke_data_2051, model_coke_2051, buying_price)
optimal_price['coke_2053'] = find_optimal_price(coke_data_2053, model_coke_2053, buying_price)
optimal_price['lemonade_2052'] = find_optimal_price(lemonade_data_2052, model_lemonade_2052, buying_price)

# 最优价格
optimal_price

# 初始价格
coke_data_2051.PRICE.describe()

<a name='8'></a>
### 结论

* 这里我们看到，最高价格为13.27时，而能实现最大利润的最优价格可能是19.15。这意味着咖啡馆在利润上存在亏损。但如果他们以19.15的价格出售，销量可能会下降，当然销售数量会减少，但利润会增加。

* 这不是在正常日子的情况，或者像节假日、有活动进行的其他日子，这些日子会对顾客的购买模式产生各种影响，通常在这些日子消费量会增加，必须对此进行特殊处理。

* 同样重要的是，要排除除价格以外可能影响顾客购买行为的任何外部因素，包括商品处于折扣状态的数据点。在这种情况下，没有商品处于折扣状态。

* 现在我们已经为每种商品确定了这些新的价格点，一旦设定，就很重要持续监控这些商品的销售和利润。可以创建一个仪表板来监控这类问题并提升利润（增加利润）